In [1]:
from funcx.sdk.client import FuncXClient
import time


fxc = FuncXClient()
fxc.throttling_enabled = False
print(fxc.base_url, fxc.throttling_enabled)

Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall&state=_default&response_type=code&code_challenge=nCbLmcqPdF5r8VcgdozLKNgIJ-cYE1rNpH6GWGXK16M&code_challenge_method=S256&access_type=offline&prefill_named_grant=FuncX+SDK+Login
Please Paste your Auth Code Below: 
320UyOyNEoRmKGmRdS4EWQEvTW0IvB
https://funcx.org/api/v1 False


# Functions to work with endpoints

In [135]:
endpoints = ['709118de-1103-463f-8425-281eb93b55ff',   #Theta covid19
             '67e95158-8bda-4b1f-a0ef-31a1626eba00',   #Ryan ep
            ]

In [136]:
def get_endpoints_status(endpoints):
    endpoint_status = {}
    idle_workers = {}
    for ep in endpoints:
        endpoint_status[ep] = fxc.get_endpoint_status(ep)[0]
        idle_workers[ep] = endpoint_status[ep]['idle_workers']
    return endpoint_status, idle_workers

# fx functions

In [137]:
def warm_endpoint():
    """A funcX function to warm the endpoint 
    and create workers"""
    pass

In [138]:
def funcx_runner(index, filename, batchsize, index_start, index_end, workdir=None, out_file=None):
    """funcX function to perform work"""
    from candle_apps.candle_node_local import funcx_node_local
    return funcx_node_local(filename, index, batchsize, index_start, index_end,
                            workdir=workdir, out_file=out_file)

In [139]:
func_uuid2 = fxc.register_function(funcx_runner,
                                   description="A funcx function for covid19")

print(func_uuid2)

87eb7d7d-51ea-470a-bc59-101d4f5ffa6f


# Gerate workloads

In [140]:
def generate_batch(filename, start=0, batchsize=10, max_batches=10):
    counter = 0
    if max_batches == 0:
        max_batches = 999999999

    x = 'Hello'
    batch_index = []
    with open(filename) as current:
        batch_index.append(current.tell())
        counter += 1

        while x and counter < max_batches:
            counter += 1
            for i in range(batchsize):
                x = current.readline()

            batch_index.append(current.tell())
        return batch_index

In [141]:
func_uuid1 = fxc.register_function(generate_batch,
                                   description="A funcx function for generating batch")

print(func_uuid1)

e8f34dac-3661-45a8-9b67-65f31bacd377


In [164]:
res1 = fxc.run(filename='/home/rchard/src/covid19/ScreenPilot/ena+db.can',
               endpoint_id='67e95158-8bda-4b1f-a0ef-31a1626eba00',
               function_id=func_uuid1)

print(res1)

9f3792b7-7388-4c00-b6fa-31d83b850d73


In [165]:
batch_index = fxc.get_result(res1)
print(batch_index)

[0, 2321, 3153, 3480, 4006, 4429, 4950, 5357, 5994, 6402]


# Distribute work

Define configs for the two eps

In [166]:
configs = {'709118de-1103-463f-8425-281eb93b55ff': {'filename': '/home/zzli/candle/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/zzli/',
                                                    'out_path': '/home/zzli/candle/ScreenPilot/'},
           '67e95158-8bda-4b1f-a0ef-31a1626eba00': {'filename': '/home/rchard/src/covid19/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/rchard/',
                                                    'out_path': '/home/rchard/src/covid19/ScreenPilot/'}
          }

Submit workloads

In [167]:
def submit_job(endpoint_uuid, idle_workers, idx):
    exec_config = configs[endpoint_uuid]
    res = fxc.run(filename=exec_config['filename'],
              index=idx,
              batchsize=10,
              index_start=0,
              index_end=1000,
              workdir=exec_config['workdir'],
              out_file=f'{exec_config["out_path"]}data-{idx}',
              endpoint_id=endpoint_uuid,
              function_id=func_uuid2)
    return res

In [168]:
def do_work(batch_index):
    task_ids = []
    # iterate over the workloads
    while len(batch_index) > 0:
        # get idle workers
        stats, idle = get_endpoints_status(endpoints)
        # submit jobs to each ep
        for ep, idle_workers in idle.items():
            # submit a job for each idle worker
            for x in range(idle_workers):
                try:
                    idx = batch_index.pop()
                    print(f'submitting {idx} to {ep}')
                    task_ids.append(submit_job(ep, idle, idx))
                except IndexError as e:
                    print('Finished!')
                    return task_ids
        time.sleep(5)
    return task_ids

In [169]:
task_ids = do_work(batch_index)

submitting 6402 to 709118de-1103-463f-8425-281eb93b55ff
submitting 5994 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 5357 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 4950 to 709118de-1103-463f-8425-281eb93b55ff
submitting 4429 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 4006 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 3480 to 709118de-1103-463f-8425-281eb93b55ff
submitting 3153 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 2321 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 0 to 709118de-1103-463f-8425-281eb93b55ff
Finished!


In [170]:
print(task_ids)

['05a89995-5fee-4bb1-bd3f-d7463960c8cd', 'c0a24889-fa33-42c7-b7e4-c2520fc1b2cf', 'baf70191-68f7-499e-a66e-f385fe577f16', 'ce3cddec-e86f-478f-87f1-d101f851f1fa', 'b8edef88-f493-4d21-9df2-d12d73ea2b62', 'bd9a7112-5e19-4b5b-95ae-fa45b3418db1', 'a68b2475-bfce-4c07-a4ed-d90c774f2f1c', '8b147e5d-3c9c-4f0e-b930-6343d159e004', '874ad256-5ef1-44a8-b9b5-53b196060587', '9d22ab08-1d72-4937-9ad7-cd2cc11680cb']


In [171]:
x = fxc.get_batch_status(task_ids)
print(x)
try:
    print(x[res[0]]['exception'].reraise())
except:
    pass

{'05a89995-5fee-4bb1-bd3f-d7463960c8cd': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-6402'}, 'c0a24889-fa33-42c7-b7e4-c2520fc1b2cf': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-5994'}, 'baf70191-68f7-499e-a66e-f385fe577f16': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-5357'}, 'ce3cddec-e86f-478f-87f1-d101f851f1fa': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-4950'}, 'b8edef88-f493-4d21-9df2-d12d73ea2b62': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-4429'}, 'bd9a7112-5e19-4b5b-95ae-fa45b3418db1': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-4006'}, 'a68b2475-bfce-4c07-a4ed-d90c774f2f1c': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-3480'}, '8b147e5d-3c9c-4f0e-b930-6343d159e004': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-3153'}, '874ad256-5ef1-44a8-b9b5-53b196060587': {'pe